# Send Data From Python to MySQL: 

**<font size=5 color=red>Mr Fugu Data Science</font>**

# (◕‿◕✿)

[Github](https://github.com/MrFuguDataScience) | [Youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw/)

`Outcome & Puporse`:

+ Connect to MySQL from Python

+ Send `.CSV()` to MySQL

+ Send `Dataframe` to MySQL

+ Send Dump `.CSV( )` from MySQL

+ Lastly, show basic useful commands

In [4]:
import mysql.connector as mysql # establish connection to mysql 

from mysql.connector import MySQLConnection # using Class for function call

import pandas as pd  # Create Dataframe

from python_to_myql_config import read_db_config # calling config files, 
# that stored user credentials

import csv

`-----------------------------------`

# Alternate Connection: `If you are NOT concerned about credentials`

`cnx = mysql.connector.connect(user='bambam', password='password',
                              host='127.0.0.1',
                              database='cowboy_doodah')
cnx.close()`

# Starting MySql from command Line:

*I have Mac, Not sure about Windows*: `mysql.server start` to stop `replace start`

*Shutdown server*: `mysqld`


*login as root*: `mysql -uroot -p`


*Checking all Users*: ` SELECT User FROM mysql.user;`

In [5]:
db_config = read_db_config() # user credentials

conn = MySQLConnection(**read_db_config()) # establish connection

mycursor = conn.cursor() # cursor, allows us to do queries,send/retrieve data


In [6]:
# CPU Purchases data:
pd.read_csv('cpu_purchase.csv').head()

,credit_card,cpu,purchase_date
0,5399-3484-4724-7187,Intel Core i1-7554K,2019-10-31
1,1630-5261-6108-7631,AMD Ryzen 1 5827X,2017-07-16
2,4435-3866-1076-3595,Intel Core i5-9457K,2019-03-24
3,3489-7099-9906-8660,AMD Ryzen 4 3401X,2019-06-15
4,8631-4500-5666-1510,Intel Core i6-7283K,2019-11-11


In [7]:
#Creating table as per requirement


sql ="""
CREATE TABLE cpu_purchase(credit_card VARCHAR(255) NOT NULL, cpu VARCHAR(255) NOT NULL,
purchase_date DATE)
"""

mycursor.execute(sql)

In [8]:
# Show tables in current database:

mycursor.execute("SHOW TABLES")
mycursor.fetchall()

[('cpu_purchase',),
 ('user_info',),
 ('user_o',),
 ('users_inf',),
 ('users_info',)]

In [42]:
# Turn this on, if you have issues sending data

mycursor.execute('SET GLOBAL local_infile=1')

# This checks if you have issues with exporting later due to security:
mycursor.execute('SHOW VARIABLES LIKE "secure_file_priv"')
mycursor.fetchall()

[('secure_file_priv', '')]

In [24]:
# Want this ON, so we can transfer data depending on how you setup Import to MySQL
mycursor.execute("SHOW GLOBAL VARIABLES LIKE 'local_infile'")
mycursor.fetchall()

[('local_infile', 'ON')]

# Send .CSV( )  to MySQL:

In [12]:
# import csv

with open('cpu_purchase.csv') as c:
    reader= csv.reader(c,delimiter=',')
    data_list=list(reader)
# data_list[1:]

mycursor.executemany("""
INSERT INTO cpu_purchase(credit_card,cpu, purchase_date) VALUES(%s,%s,%s)
""",data_list[1:])

In [14]:

mycursor.execute("SELECT count(*) from cpu_purchase ")
mycursor.fetchall()


mycursor.execute("SELECT * from cpu_purchase LIMIT 4")
mycursor.fetchall()

[('5399-3484-4724-7187', 'Intel Core i1-7554K', datetime.date(2019, 10, 31)),
 ('1630-5261-6108-7631', 'AMD Ryzen 1 5827X', datetime.date(2017, 7, 16)),
 ('4435-3866-1076-3595', 'Intel Core i5-9457K', datetime.date(2019, 3, 24)),
 ('3489-7099-9906-8660', 'AMD Ryzen 4 3401X', datetime.date(2019, 6, 15))]

# Send Datafrane to MySQL:

In [16]:
cpu_sales= pd.read_csv('cpu_purchase.csv')

mycursor.executemany("""
INSERT INTO cpu_purchase(credit_card,cpu, purchase_date) VALUES(%s,%s,%s)
""",list(cpu_sales.values))


In [17]:
mycursor.execute('select * from cpu_purchase ')
len(mycursor.fetchall())

11652

# Export File from MySQL as .CSV()

+ There will be an error that will occur when you try to send this file.

Since, there is a `--secure-file-priv` error that will show up. This is because mysql setup it up this way. You need to do a work around.



In [32]:
'''
Run this command to see what is going on with secure-file-priv

If the printout says NULL, then you need to create a file to bypass this

Otherwise it is an empty string.
'''


mycursor.execute('SHOW VARIABLES LIKE "secure_file_priv"')
mycursor.fetchall()

[('secure_file_priv', '')]

**Create this file in the terminal it will be very short and sweet**:
If using VIM: otherwise use your editor of choice and see if it works

`vi ~/.my.cnf`

Then do: 


`[mysqld]`

`secure_file_priv               = ''`



save this file by doing 

`esc`

then

`:wq`


*check out the link in the citations for this!*

In [ ]:
# mycursor.execute("SHOW PROCESSLIST")
# mycursor.fetchall()

In [18]:
# OutFILE:

outfile="""
SELECT * FROM cpu_purchase INTO OUTFILE '~/Desktop/mysql_python/cpu_outfile_sales.csv'
"""

mycursor.execute(outfile)

In [19]:
# Check if data is still there!
mycursor.execute('select * from cpu_purchase limit 5')
mycursor.fetchall()

[('5399-3484-4724-7187', 'Intel Core i1-7554K', datetime.date(2019, 10, 31)),
 ('1630-5261-6108-7631', 'AMD Ryzen 1 5827X', datetime.date(2017, 7, 16)),
 ('4435-3866-1076-3595', 'Intel Core i5-9457K', datetime.date(2019, 3, 24)),
 ('3489-7099-9906-8660', 'AMD Ryzen 4 3401X', datetime.date(2019, 6, 15)),
 ('8631-4500-5666-1510', 'Intel Core i6-7283K', datetime.date(2019, 11, 11))]

# Basic Useful Commands:

In [20]:
# Find all Users:
mycursor.execute("SELECT User FROM mysql.user")
mycursor.fetchall()

[('MrFugu',),
 ('mysql.infoschema',),
 ('mysql.session',),
 ('mysql.sys',),
 ('root',)]

In [29]:
# Show Databases:
mycursor.execute("SHOW databases")
mycursor.fetchall()

[('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('project_practice',),
 ('sys',)]

In [30]:
# show what database we are currently using:
mycursor.execute("SELECT DATABASE()")
mycursor.fetchall()

[('project_practice',)]

In [31]:
# Show tables in current database:
mycursor.execute("SHOW TABLES")
mycursor.fetchall()

[('cpu_purchase',),
 ('user_info',),
 ('user_o',),
 ('users_inf',),
 ('users_info',)]

# Citations and Help:

# ◔̯◔

https://dev.mysql.com/doc/mysql-shell/8.0/en/mysql-shell-commands.html (mysql shell commands)

https://www.mysqltutorial.org/import-csv-file-mysql-table/

https://medium.com/@AviGoom/how-to-import-a-csv-file-into-a-mysql-database-ef8860878a68

https://dev.mysql.com/doc/connector-python/en/connector-python-example-ddl.html

https://www.real-world-systems.com/docs/date.1.html

https://michaelrigart.be/export-directly-mysql-csv/

https://medium.com/@andrewpongco/solving-the-mysql-server-is-running-with-the-secure-file-priv-option-so-it-cannot-execute-this-d319de864285 (FIX the problem exporting in MySQL)